### makemore: lect01

#### 20240227-061500

In [2]:
import torch

In [4]:
a = torch.zeros((3, 5))

In [5]:
a

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [7]:
a.dtype

torch.float32

In [8]:
a = torch.zeros((3, 5), dtype=torch.int32)

In [9]:
a

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)